# LLM with RCA after anomaly detection using RAG<br>
## Project Overview<br>
Author: Sedat Kaymaz<br>
This project aims to provide a root cause analsys method by using LLM with Dynamic RAG based on a system log file <br>
for reference after applying anomaly detection ML method to the basic telecom metric list   <br>

In [ ]:
# Run once only
#%pip install -r requirements.txt

In [1]:
import os,sys
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader, TextLoader
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.text_splitter import CharacterTextSplitter


# Load environment variables
load_dotenv()

False

In [2]:
# LLM Configuration
def get_llm(model):
    """
    Retrieves the Language Model (LLM) for root cause analysis.

    This function retrieves the Language Model (LLM) used for root cause analysis. It first checks if the OpenAI API key is set as an environment variable. If not, it prompts the user to enter the API key and sets it as an environment variable. The function then returns an instance of the ChatOpenAI class with a temperature of 0 and the model name set to "gpt-4".

    Returns:
        ChatOpenAI: An instance of the ChatOpenAI class representing the Language Model (LLM) for root cause analysis.
    """
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        openai_api_key = input("Please enter your OpenAI API key: ")
        os.environ["OPENAI_API_KEY"] = openai_api_key
    
    # You can modify this to use different models or local LLMs
    return ChatOpenAI(temperature=0, model_name=model)

print("Loading Language Model...")
model="gpt-4"
llm = get_llm(model)
print(f"Language Model {model} loaded.")

Loading Language Model...
Language Model gpt-4 loaded.


In [10]:

# RAG for reading and processing metrics file
def process_metrics_file(filename):
    """
    Process the metrics file and return a vector store.

    Args:
        filename (str): The name of the metrics file.

    Returns:
        vectorstore: A vector store containing the embeddings of the text documents.
    """
    loader = CSVLoader(f"data/{filename}")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(texts, embeddings)
    return vectorstore

# Process metrics file
metrics_vectorstore = process_metrics_file("metrics.csv")

# Load metrics data for anomaly detection
df = pd.read_csv("data/metrics.csv")
df['time'] = pd.to_datetime(df['time'])
print(df.head(5))

                 time  call_attempt  call_success  call_failure  \
0 2024-09-04 00:00:00           114           110             0   
1 2024-09-04 00:01:00           113           110             0   
2 2024-09-04 00:02:00           114           111             0   
3 2024-09-04 00:03:00           113           111             1   
4 2024-09-04 00:04:00           112           111             1   

   total_registered_subs  call_success_rate  
0                   9031              96.40  
1                   9084              97.34  
2                   9089              97.36  
3                   9035              98.23  
4                   9092              99.10  


In [9]:
# Anomaly detection using Isolation Forest
def detect_anomalies(df):
    """
    Detects anomalies in the given DataFrame using the Isolation Forest algorithm.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame containing the data to be analyzed.

    Returns:
    - pandas.DataFrame: A subset of the input DataFrame containing only the rows that are classified as anomalies.
    """

    features = ['call_attempt', 'call_success', 'call_failure', 'total_registered_subs', 'call_success_rate']
    X = df[features]
    
    iso_forest = IsolationForest(contamination=0.005, random_state=42)
    anomalies = iso_forest.fit_predict(X)
    
    df['is_anomaly'] = anomalies
    return df[df['is_anomaly'] == -1]

# Detect anomalies
anomalies = detect_anomalies(df)
if anomalies.empty:
    print("No anomalies detected in the metrics.")

print(f"Anomalies found:\n{anomalies}\n")

Anomalies found:
                   time  call_attempt  call_success  call_failure  \
683 2024-09-04 11:23:00           114            27             0   
684 2024-09-04 11:24:00           113            32             0   
685 2024-09-04 11:25:00           112            40             0   
686 2024-09-04 11:26:00           114            70             2   

     total_registered_subs  call_success_rate  is_anomaly  
683                   9029             23.491          -1  
684                   9038             28.368          -1  
685                   9033             35.730          -1  
686                   9157             61.491          -1  



In [7]:
# RAG for processing log file
def process_log_file(filename):
    """
    Process a log file and return a vector store.

    Args:
        filename (str): The name of the log file to process.

    Returns:
        vectorstore: A vector store containing embeddings of the log file texts.
    """
    loader = TextLoader(f"data/{filename}")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(texts, embeddings)
    return vectorstore


# Process log file
print("Processing log file...")
logs_vectorstore = process_log_file("systemd.log")
print("Log file has been processed.")   

Processing log file...
Log file has been processed.


In [8]:

# Root cause analysis using LLM
def analyze_root_cause(llm, metrics_vectorstore, logs_vectorstore, anomalies):
    """
    Analyzes the root cause of anomalies in the metrics based on the system logs.

    Args:
        llm (LLM): The LLM object used for analysis.
        metrics_vectorstore (VectorStore): The vector store containing metrics information.
        logs_vectorstore (VectorStore): The vector store containing log information.
        anomalies (list): A list of anomalies to be analyzed.

    Returns:
        str: A detailed root cause analysis for the given anomalies.

    Example:
        anomalies = ["Anomaly1", "Anomaly2"]
        result = analyze_root_cause(llm, metrics_vectorstore, logs_vectorstore, anomalies)
        print(result)
    """
    
    prompt_template = """
    Analyze the following anomalies in the metrics and provide a root cause analysis based on the system logs:

    Anomalies:
    {anomalies}

    Relevant metrics information:
    {metrics_info}

    Relevant log information:
    {logs_info}

    Please provide a detailed root cause analysis for these anomalies.
    """

    prompt = PromptTemplate(
        input_variables=["anomalies", "metrics_info", "logs_info"],
        template=prompt_template
    )

    # Create a RunnableSequence
    chain = (
        {
            "anomalies": RunnablePassthrough(),
            "metrics_info": lambda x: metrics_vectorstore.similarity_search(str(x), k=2),
            "logs_info": lambda x: logs_vectorstore.similarity_search(str(x), k=2)
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    # Invoke the chain
    result = chain.invoke(anomalies)
    
    return result


# Perform root cause analysis
analysis = analyze_root_cause(llm, metrics_vectorstore, logs_vectorstore, anomalies)

print("Root Cause Analysis:")
print(analysis)


Root Cause Analysis:
Based on the provided logs and metrics, the anomalies in the metrics seem to be related to the OpenStack services, specifically the Open vSwitch service and the Nova Compute service.

At 11:22:13, there is a log entry indicating an assertion failure in the Open vSwitch service, which leads to the service being killed and restarted. This could potentially disrupt network connectivity for the OpenStack services, affecting call attempts and successes.

The Nova Compute service logs show several instances being migrated, rebooted, created, and shut down around the same time. This could potentially cause disruptions in the service, affecting the call success rate. Specifically, at 11:23:01, there is a log entry indicating the start of a migration for an instance, which could potentially disrupt the service.

In addition, the total number of registered subscribers increases from 9033 to 9157 between 11:25:00 and 11:26:00. This sudden increase could potentially overload t